# Smart Aquaculture Use Case for the Computate Smart Cloud Builder

## About the open source GPL3 license and copyright for this product

Copyright © 2024 Computate Limited Liability Company in Utah, USA

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.

ADDITIONAL TERMS

As stated in section 7. c) and e) of the GPL3 license, 
"you may supplement the terms of this License with terms," 
Computate has added the following additional terms to the license: 

  7 c) Prohibiting misrepresentation of the origin of that material, and
    requiring that modified versions of such material be marked in
    reasonable ways as different from the original version;

  7 e) Declining to grant rights under trademark law for use of some
    trade names, trademarks, or service marks;

Please do not redistribute this use case unless you have built your own fork 
of this use case with these tools, separate from this use case, 
and customized for your organization. 

These terms allow you to monetize your own customized open source use case 
for your own organization in a sustainable open source way, and pull in 
upstream changes from this use case into your customized use case. 

QUESTIONS

For questions about this open source license, 
please contact our public mailing list at computate@group.computate.org

# Load the code search model into the Solr search engine

In this hackathon, we will be generating code with AI/ML provided by the well established [computate project](https://github.com/computate-org/computate). We will automatically generate OpenAPI specs, database table schemas, Java POJOs, Vert.x Reactive Java APIs, Handlebars HTML page templates, JavaScript page and API functions, NGSI-LD Context data, OpenShift Custom Resource Definitions, and Ansible Operator roles and playbooks for any of the hundreds of open source Edge device data related [FIWARE Smart Data Models available here](https://github.com/smart-data-models). 

## Clone and index all Smart Data Models

Run the Ansible Playbook below to clone the hundreds of FIWARE Smart Data Models, and index each Smart Data Model in Apache Solr. We search this Smart Data Model data when we create new Java classes to see if there is a `SmartDataModel: search terms...` class comment in the code that closely matches a Smart Data Model in the Solr search engine, then we will generate all the code for this Smart Data Model automatically. 

In [ ]:
eval $(cd ../../ && ./vars.py)
(cd ../../ && ansible-playbook -e @$VARS_PATH playbooks/clone-smart-model-data.yaml)
echo DONE

## Search for Smart Data Models

Query the search engine for a smart data model matching the `SEARCH_TERM`. 
You can customize the `SEARCH_TERM` environment variable below for different results. 

In [ ]:
eval $(cd ../../ && ./vars.py)
SEARCH_TERM="Fish Feed"
REGEX1='s/( +)/ OR /g'
REGEX2='s/^([\w\W])$/"$1"/g'
ENCODED_STR="$(echo $SEARCH_TERM | perl -p -e 's/([a-z])([A-Z])/$1 $2/g' | perl -p -e 's/( +)/" OR "/g' | perl -p -e 's/^([\w\W]+?)\s*$/("$1")/g')"
echo $ENCODED_STR
DOMAIN_QUERY="domainName_suggested:$ENCODED_STR^2 OR domainName_text_enUS:$ENCODED_STR^5"
SUBMODULE_QUERY="submoduleShortName_suggested:$ENCODED_STR^3 OR submoduleShortName_text_enUS:$ENCODED_STR^10"
MODEL_QUERY="modelName_suggested:$ENCODED_STR^10 OR modelName_text_enUS:$ENCODED_STR^20"
SEARCH_QUERY="$DOMAIN_QUERY OR $SUBMODULE_QUERY OR $MODEL_QUERY"
STUFF="&fl=*,score&sort=score desc"
echo $SEARCH_QUERY && echo

curl -k -s -u "$SOLR_USERNAME:$SOLR_PASSWORD" "$SOLR_URL_COMPUTATE"'/query' \
    --data-urlencode 'fl=*,score' \
    --data-urlencode 'sort=score desc' \
    --data-urlencode "fq=computateFiwareSmartDataModel_docvalues_boolean:true" \
    --data-urlencode "q=$SEARCH_QUERY" \
  | jq -r '"\(.response.docs | map(.domainName_docvalues_string + " " + .submoduleShortName_docvalues_string + " " + .modelName_docvalues_string) | join("\n"))"'
echo DONE